<a href="https://colab.research.google.com/github/OlgaPuntous/HPA-single-cell-image-classification/blob/master/hpa_rgb_model_rgby_classif_binaire_olga.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intro

Training CV model example with pre-trained weights. Data from https://www.kaggle.com/c/hpa-single-cell-image-classification. Data represents 4 channnels images ( RED, BLUE, GREEN, YELLOW), each picture could contain un to 18 labels. In some domains training the CV data with more than 3 channels could be actuality (satelite images, medical pictures)

Considering the size of training data, learning a deep 4-channel model with weights initialized at random might be problematic. But all ImageNet-pre-trained models have 3-channels.

This notebook demonstrates how to initialize a 4-channel EfficientNet with weights reused from a pre-trained 3-channel model.

For simplification data choosed for training is set of pictures from https://www.kaggle.com/c/hpa-single-cell-image-classification, which contains only one label.Some data was droped to avid disbalance of classes. The predictions shows if test set contains label 1 or not.


### Libraries

In [ ]:
import gc
gc.collect()

56

In [ ]:
!pip install "../input/keras-application/Keras_Applications-1.0.8-py3-none-any.whl"
!pip install "../input/efficientnet111/efficientnet-1.1.1-py3-none-any.whl"

Processing /kaggle/input/keras-application/Keras_Applications-1.0.8-py3-none-any.whl
Keras-Applications is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
Processing /kaggle/input/efficientnet111/efficientnet-1.1.1-py3-none-any.whl
efficientnet is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.


In [ ]:
import os, glob
import tensorflow as tf
from tensorflow.keras.models import Sequential
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
import random
from sklearn.model_selection import train_test_split
import cv2
import numpy as np
import pandas as pd
import multiprocessing
from copy import deepcopy
import keras
import keras.backend as K
from efficientnet.keras import EfficientNetB0
from keras.optimizers import Adam
from keras.callbacks import Callback
from keras.layers import Dense, Flatten
from keras.models import Model, load_model
from keras.utils import Sequence
from albumentations import Compose, VerticalFlip, HorizontalFlip, Rotate, GridDistortion
import matplotlib.pyplot as plt
from IPython.display import Image, display
from numpy.random import seed
from tqdm.auto import tqdm
from tensorflow.python.framework import ops
from subprocess import check_output
import matplotlib.pyplot as plt 

tf.random.set_seed(10)
%matplotlib inline

# 4-channel classifier init

In [ ]:
TEST_IMGS_FOLDER = '../input/hpa-single-cell-image-classification/test/'
TRAIN_IMGS_FOLDER = '../input/hpa-single-cell-image-classification/train/'
IMG_HEIGHT = IMG_WIDTH = 128
BATCH_SIZE = 64

DOWNLOAD_PRETRAINED_WEIGHTS = False

In [ ]:

from sklearn.preprocessing import MultiLabelBinarizer
def binarize_data(df):
    mlb = MultiLabelBinarizer()
    df['labels_list']=[list(map(int, i.split("|"))) for i in df.Label]
    mlb.fit(df['labels_list'].to_list())
    Labels_binarized = mlb.transform(df['labels_list'])
    return Labels_binarized

#Version avec le resize
def open_rgby(image_id): #a function that reads RGBY image with resizing
    colors = ['red','green','blue','yellow']
    img = [cv2.imread(os.path.join(TRAIN_IMGS_FOLDER, f'{image_id}_{color}.png'), cv2.IMREAD_GRAYSCALE) for color in colors]
    img = np.stack(img, axis=-1)
    img_resized = cv2.resize(img, (128, 128))
    img_resized=img_resized/255
    return img_resized

def label_choice (n):
     data_train['Label_{}'.format(n)]=data_train['Multilabel'].apply(lambda x:x[n])
        
def dataset_split(n): #n - label number
    
    y=data_train_croped['Label_{}'.format(n)]
    X=data_train_croped["ID"]

    X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=42,stratify=y)
    
    return (X_train, X_test, y_train, y_test)

def data_aug(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_contrast(image, 0.2, 0.5)
    image = tf.image.random_crop(image, [128, 128, 4])
    image=tf.image.rot90(image, k=1, name=None)
    return image, label

''' SELECT SUB DATASET WITH ONLY LABEL(n) = 1'''
def data_monolabel_sub(n):
    label='Label_{}'.format(n)
    mask=(data_monolabel[label]==1)
    data_sub = data_monolabel.loc[mask]
    return(data_sub)

''' SELECT SUB DATASET of EQUAL SIZE WITH ONLY LABEL(n) = 0'''
def data_monolabel_sub_nolabel(n):
    label='Label_{}'.format(n)
    mask=(data_monolabel[label]==0)
    data_sub_nolabel = data_monolabel.loc[mask]
    data_sub_nolabel = data_monolabel.sample(len(data_monolabel_sub(n)), random_state=42)
    return(data_sub_nolabel)  

''' SELECT SUB DATASET WITH ONLY LABEL(n) = 1'''
def data_monolabel_sub(n):
    label='Label_{}'.format(n)
    mask=(data_monolabel[label]==1)
    data_sub = data_monolabel.loc[mask]
    return(data_sub)

''' SELECT SUB DATASET of EQUAL SIZE WITH ONLY LABEL(n) = 0'''
def data_monolabel_sub_nolabel(n):
    label='Label_{}'.format(n)
    mask=(data_monolabel[label]==0)
    data_sub_nolabel = data_monolabel.loc[mask]
    data_sub_nolabel = data_monolabel.sample(len(data_monolabel_sub(n)), random_state=42)
    return(data_sub_nolabel)

# Pre-processing data labels before training/CODE CD multibinarizer

In [ ]:
data_train = pd.read_csv("../input/hpa-single-cell-image-classification/train.csv")
data_test=pd.read_csv("../input/hpa-single-cell-image-classification/sample_submission.csv")
data_train['labels_list']=[list(map(int, i.split("|"))) for i in data_train.Label]

data_train.head()

,ID,Label,labels_list
0,5c27f04c-bb99-11e8-b2b9-ac1f6b6435d0,8|5|0,"[8, 5, 0]"
1,5fb643ee-bb99-11e8-b2b9-ac1f6b6435d0,14|0,"[14, 0]"
2,60b57878-bb99-11e8-b2b9-ac1f6b6435d0,6|1,"[6, 1]"
3,5c1a898e-bb99-11e8-b2b9-ac1f6b6435d0,16|10,"[16, 10]"
4,5b931256-bb99-11e8-b2b9-ac1f6b6435d0,14|0,"[14, 0]"


In [ ]:
multilabel=binarize_data(data_train)
multilabel_list = multilabel.tolist()
data_train['Multilabel']=[x for x in multilabel_list]
data_train['Multilabel']=data_train['Multilabel'].apply(lambda x:x[0:len(x)-1])

[label_choice(i) for i in range(12)]

data_train.head()


,ID,Label,labels_list,Multilabel,Label_0,Label_1,Label_2,Label_3,Label_4,Label_5,Label_6,Label_7,Label_8,Label_9,Label_10,Label_11
0,5c27f04c-bb99-11e8-b2b9-ac1f6b6435d0,8|5|0,"[8, 5, 0]","[1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...",1,0,0,0,0,1,0,0,1,0,0,0
1,5fb643ee-bb99-11e8-b2b9-ac1f6b6435d0,14|0,"[14, 0]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...",1,0,0,0,0,0,0,0,0,0,0,0
2,60b57878-bb99-11e8-b2b9-ac1f6b6435d0,6|1,"[6, 1]","[0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,1,0,0,0,0,1,0,0,0,0,0
3,5c1a898e-bb99-11e8-b2b9-ac1f6b6435d0,16|10,"[16, 10]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...",0,0,0,0,0,0,0,0,0,0,1,0
4,5b931256-bb99-11e8-b2b9-ac1f6b6435d0,14|0,"[14, 0]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...",1,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
data_train["MonoLabel"]=data_train['Multilabel'].apply(lambda x:1 if sum(x)==1 else 0)
data_monolabel=data_train[data_train["MonoLabel"]==1] #changed for 11 label
data_multilabel=data_train[data_train["MonoLabel"]==0]
data_multilabel=data_multilabel.sample(100)
data_train

,ID,Label,labels_list,Multilabel,Label_0,Label_1,Label_2,Label_3,Label_4,Label_5,Label_6,Label_7,Label_8,Label_9,Label_10,Label_11,MonoLabel
0,5c27f04c-bb99-11e8-b2b9-ac1f6b6435d0,8|5|0,"[8, 5, 0]","[1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...",1,0,0,0,0,1,0,0,1,0,0,0,0
1,5fb643ee-bb99-11e8-b2b9-ac1f6b6435d0,14|0,"[14, 0]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...",1,0,0,0,0,0,0,0,0,0,0,0,0
2,60b57878-bb99-11e8-b2b9-ac1f6b6435d0,6|1,"[6, 1]","[0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,1,0,0,0,0,1,0,0,0,0,0,0
3,5c1a898e-bb99-11e8-b2b9-ac1f6b6435d0,16|10,"[16, 10]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...",0,0,0,0,0,0,0,0,0,0,1,0,0
4,5b931256-bb99-11e8-b2b9-ac1f6b6435d0,14|0,"[14, 0]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...",1,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21801,dd0989c4-bbca-11e8-b2bc-ac1f6b6435d0,14,[14],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...",0,0,0,0,0,0,0,0,0,0,0,0,1
21802,dd1f7fb8-bbca-11e8-b2bc-ac1f6b6435d0,3|0,"[3, 0]","[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,0,0,1,0,0,0,0,0,0,0,0,0
21803,dd5cb36a-bbca-11e8-b2bc-ac1f6b6435d0,14|0,"[14, 0]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...",1,0,0,0,0,0,0,0,0,0,0,0,0
21804,df573730-bbca-11e8-b2bc-ac1f6b6435d0,14,[14],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...",0,0,0,0,0,0,0,0,0,0,0,0,1


In [ ]:
n=1 # training will be performed on monolabel data with 1 label
data_train_croped = pd.concat([data_monolabel_sub(n),data_monolabel_sub_nolabel(n)], ignore_index=True)
data_train_croped.shape

(442, 17)

In [ ]:
data_train_croped

,ID,Label,labels_list,Multilabel,Label_0,Label_1,Label_2,Label_3,Label_4,Label_5,Label_6,Label_7,Label_8,Label_9,Label_10,Label_11,MonoLabel
0,eb7caa62-bb99-11e8-b2b9-ac1f6b6435d0,1,[1],"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,1,0,0,0,0,0,0,0,0,0,0,1
1,b829943a-bb9a-11e8-b2b9-ac1f6b6435d0,1,[1],"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,1,0,0,0,0,0,0,0,0,0,0,1
2,0881d08c-bb9b-11e8-b2b9-ac1f6b6435d0,1,[1],"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,1,0,0,0,0,0,0,0,0,0,0,1
3,136873e8-bb9b-11e8-b2b9-ac1f6b6435d0,1,[1],"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,1,0,0,0,0,0,0,0,0,0,0,1
4,22dbcd20-bb9b-11e8-b2b9-ac1f6b6435d0,1,[1],"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,1,0,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
437,2d8fc63c-bba4-11e8-b2b9-ac1f6b6435d0,15,[15],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,0,0,0,0,0,0,0,0,0,0,0,1
438,dd3440de-bba1-11e8-b2b9-ac1f6b6435d0,6,[6],"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,0,0,0,0,0,1,0,0,0,0,0,1
439,d76f2976-bbc3-11e8-b2bc-ac1f6b6435d0,5,[5],"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,0,0,0,0,1,0,0,0,0,0,0,1
440,0dd56ec6-bb9c-11e8-b2b9-ac1f6b6435d0,0,[0],"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,0,0,0,0,0,0,0,0,0,0,0,1


In [ ]:
X_train_0, X_test_0, y_train_0, y_test_0=dataset_split(n)


"''\nX_train_1, X_test_1, y_train_1, y_test_1=dataset_split(1)\nX_train_2, X_test_2, y_train_2, y_test_2=dataset_split(2)\nX_train_3, X_test_3, y_train_3, y_test_3=dataset_split(3)\nX_train_4, X_test_4, y_train_4, y_test_4=dataset_split(4)\nX_train_5, X_test_5, y_train_5, y_test_5=dataset_split(5)\n\n"

In [ ]:
ds_train_0 = tf.data.Dataset.from_tensor_slices([(open_rgby(str(x))) for x in tqdm(X_train_0)])

  0%|          | 0/353 [00:00<?, ?it/s]

"''\n\npath_save='./train_ds_0'\ntf.data.experimental.save(\n    ds_train_0, path_save, compression=None, shard_func=None)\n"

In [ ]:
ds_val_0 = tf.data.Dataset.from_tensor_slices([(open_rgby(str(x))) for x in tqdm(X_test_0)])

  0%|          | 0/89 [00:00<?, ?it/s]

In [ ]:
ds_label_0=tf.data.Dataset.from_tensor_slices(y_train_0)

ds_label_val_0=tf.data.Dataset.from_tensor_slices(y_test_0)

In [ ]:
ds_full_0 = tf.data.Dataset.zip((ds_train_0,ds_label_0))

ds_full_val_0 = tf.data.Dataset.zip((ds_val_0,ds_label_val_0))

In [ ]:
repeated_full_ds = ds_full_0.repeat(10)

repeated_full_ds =repeated_full_ds.map(data_aug)



In [ ]:
repeated_ds_full_val = ds_full_val_0.repeat(10)

repeated_ds_full_val = repeated_ds_full_val.map(data_aug)

In [ ]:
repeated_full_ready_ds=repeated_full_ds.shuffle(len(repeated_full_ds)).batch(32)

ds_full_val_ready_0=repeated_ds_full_val.batch(32)

## A model with weights pre-trained on ImageNet

In [ ]:
# you'll need an internet connection to download ImageNet weights,
# for illustration I'm using a randomly generated RGB model
weights_init = 'imagenet' if DOWNLOAD_PRETRAINED_WEIGHTS else None

imagenet_model = EfficientNetB0(weights=weights_init, include_top=False, pooling='avg',
                               input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))
rgb_model_output = Dense(1, activation='sigmoid')(imagenet_model.output)
model_rgb = Model(inputs=imagenet_model.input, outputs=rgb_model_output)

## A RGBY model

In [ ]:
four_channel_effnet = EfficientNetB0(weights=None, include_top=False, pooling='avg', 
                                     input_shape=(IMG_HEIGHT, IMG_WIDTH, 4)) # base model
model_rgby_output = Dense( 1, activation='sigmoid')(four_channel_effnet.output)
model_rgby = Model(inputs=four_channel_effnet.input, outputs=model_rgby_output)
model_rgby.summary()

/opt/conda/lib/python3.7/site-packages/keras_applications/imagenet_utils.py:279: UserWarning: This model usually expects 1 or 3 input channels. However, it was passed an input_shape with 4 input channels.
  str(input_shape[-1]) + ' input channels.')


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 128, 128, 4) 0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 64, 64, 32)   1152        input_2[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 64, 64, 32)   128         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 64, 64, 32)   0           stem_bn[0][0]                    
____________________________________________________________________________________________

In [ ]:
link_imagenet = '../input/efficientnet-keras-dataset/weights/efficientnet-b0_weights_tf_dim_ordering_tf_kernels_autoaugment_notop.h5'
imagenet_model = EfficientNetB0(weights= link_imagenet, include_top=False, pooling='avg', 
                                     input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))

In [ ]:
imagenet_model.summary()

Model: "efficientnet-b0"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 64, 64, 32)   864         input_3[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 64, 64, 32)   128         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 64, 64, 32)   0           stem_bn[0][0]                    
____________________________________________________________________________________

In [ ]:
rgb_model_output = Dense(1, activation='sigmoid')(imagenet_model.output)
model_rgb = Model(inputs=imagenet_model.input, outputs=rgb_model_output)

In [ ]:
for layer in tqdm(model_rgby.layers, desc='Copying the pre-trained net weights..'):
    if 'input' in layer.name or 'dense' in layer.name:
        continue
    elif layer.name == 'stem_conv':
#         with graph_green.as_default():
        kernels = model_rgb.get_layer('stem_conv').get_weights()[0]
        kernels_extra_channel = np.concatenate((kernels, kernels[:,:,:1,:]), axis=-2)
        layer.set_weights([kernels_extra_channel])
    else:
#         with graph_green.as_default():
        weights_green = model_rgb.get_layer(layer.name).get_weights()
        layer.set_weights(weights_green)

Copying the pre-trained net weights..:   0%|          | 0/232 [00:00<?, ?it/s]

In [ ]:
four_channel_effnet.trainable = False

In [ ]:
# Let's create a learning rate schedule to decrease the learning rate as we train the model. 
initial_learning_rate = 0.0001

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=1000,
    decay_rate=0.96,
    staircase=True
)

In [ ]:
model_rgby.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = lr_schedule),
              loss = tf.keras.losses.BinaryCrossentropy(),
              metrics = [tf.keras.metrics.BinaryAccuracy()]
              )

In [ ]:
history=model_rgby.fit(repeated_full_ready_ds, epochs=30, validation_data=ds_full_val_ready_0) # only understand 1 input tensor e.g. list with 1 array 
print("\n ------ \n","training history:",history.history)

Epoch 1/30
111/111 [==============================] - 17s 52ms/step - loss: 0.6653 - binary_accuracy: 0.6039 - val_loss: 0.6352 - val_binary_accuracy: 0.6596
Epoch 2/30
111/111 [==============================] - 8s 35ms/step - loss: 0.6406 - binary_accuracy: 0.6444 - val_loss: 0.6190 - val_binary_accuracy: 0.6820
Epoch 3/30
111/111 [==============================] - 7s 34ms/step - loss: 0.6229 - binary_accuracy: 0.6583 - val_loss: 0.6098 - val_binary_accuracy: 0.6921
Epoch 4/30
111/111 [==============================] - 7s 34ms/step - loss: 0.6154 - binary_accuracy: 0.6629 - val_loss: 0.6016 - val_binary_accuracy: 0.6809
Epoch 5/30
111/111 [==============================] - 7s 34ms/step - loss: 0.6044 - binary_accuracy: 0.6792 - val_loss: 0.5944 - val_binary_accuracy: 0.6899
Epoch 6/30
111/111 [==============================] - 8s 34ms/step - loss: 0.5939 - binary_accuracy: 0.6893 - val_loss: 0.5882 - val_binary_accuracy: 0.6910
Epoch 7/30
111/111 [==============================] - 7s 

In [ ]:
from plotly import graph_objects as go
color_chart = ["#4B9AC7", "#4BE8E0", "#9DD4F3", "#97FBF6", "#2A7FAF", "#23B1AB", "#0E3449", "#015955"]
fig = go.Figure(data=[
                      go.Scatter(
                          y=history.history["binary_accuracy"],
                          name="Training accuracy",
                          mode="lines",
                          marker=dict(
                              color=color_chart[4]
                          )),
                      go.Scatter(
                          y=history.history["val_binary_accuracy"],
                          name="Validation accruracy",
                          mode="lines",
                          marker=dict(
                              color=color_chart[5]
                          ))
])
fig.update_layout(
    title='Training and val accuracy across epochs',
    xaxis_title='epochs',
    yaxis_title='Accuracy'    
)
fig.show()


In [ ]:
from plotly import graph_objects as go
fig = go.Figure(data=[
                      go.Scatter(
                          y=history.history["loss"],
                          name="Training loss",
                          mode="lines",
                          marker=dict(
                              color=color_chart[0]
                          )),
                      go.Scatter(
                          y=history.history["val_loss"],
                          name="Validation loss",
                          mode="lines",
                          marker=dict(
                              color=color_chart[1]
                          ))
])
fig.update_layout(
    title='Training and val loss across epochs',
    xaxis_title='epochs',
    yaxis_title='Cross Entropy'    
)
fig.show()


In [ ]:
'''EXECUTE'''
model_rgby.save("./SAVE_MODEL/model_label1.h5")
print("Done--------")

Done--------


In [ ]:
data_multilabel['prediction']=data_multilabel['ID'].apply(lambda x: model_rgby.predict(np.expand_dims((open_rgby2(str(x))),0)))

#predictions_test = model_rgby.predict(np.expand_dims((open_rgby2(str(x))) for x in tqdm(data_multilabel['ID']), 0))

In [ ]:
data_multilabel['prediction_lb']=data_multilabel['prediction'].apply(lambda x: 1 if x>0.5 else 0)
                                                                                                    
data_multilabel['r']=data_multilabel['prediction_lb']-data_multilabel['Label_0']
data_multilabel

,ID,Label,labels_list,Multilabel,Label_0,Label_1,Label_2,Label_3,Label_4,Label_5,Label_6,Label_7,Label_8,Label_9,Label_10,Label_11,MonoLabel,prediction,prediction_lb,r
7126,0e4004da-bbaa-11e8-b2ba-ac1f6b6435d0,3|0,"[3, 0]","[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,0,0,1,0,0,0,0,0,0,0,0,0,[[0.46446425]],0,-1
6756,2e3af156-bba9-11e8-b2ba-ac1f6b6435d0,1|0,"[1, 0]","[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,1,0,0,0,0,0,0,0,0,0,0,0,[[0.38429052]],0,-1
2983,86a158de-bba0-11e8-b2b9-ac1f6b6435d0,16|0,"[16, 0]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,0,0,0,0,0,0,0,0,0,0,0,0,[[0.37283778]],0,-1
5394,3c85e25a-bba6-11e8-b2ba-ac1f6b6435d0,12|0,"[12, 0]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...",1,0,0,0,0,0,0,0,0,0,0,0,0,[[0.26895607]],0,-1
19466,d786462c-bbc5-11e8-b2bc-ac1f6b6435d0,16|0,"[16, 0]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,0,0,0,0,0,0,0,0,0,0,0,0,[[0.87434834]],1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2763,f902d052-bb9f-11e8-b2b9-ac1f6b6435d0,1|0,"[1, 0]","[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,1,0,0,0,0,0,0,0,0,0,0,0,[[0.6302632]],1,0
8873,d839159e-bbad-11e8-b2ba-ac1f6b6435d0,2|0,"[2, 0]","[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,0,1,0,0,0,0,0,0,0,0,0,0,[[0.41904634]],0,-1
49,7465a65e-bb99-11e8-b2b9-ac1f6b6435d0,12|0,"[12, 0]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...",1,0,0,0,0,0,0,0,0,0,0,0,0,[[0.35606942]],0,-1
13180,80dfd562-bbb7-11e8-b2ba-ac1f6b6435d0,5|0,"[5, 0]","[1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,0,0,0,0,1,0,0,0,0,0,0,0,[[0.15726769]],0,-1


In [ ]:
data_multilabel['r'].value_counts()

 0    47
-1    44
 1     9
Name: r, dtype: int64

In [ ]:
data_multilabel['r'].value_counts()

 0    47
-1    44
 1     9
Name: r, dtype: int64